In [1]:
"""
@author: Sina Zarrieß
"""

import pandas as pd
import glob
import os
import json
import sys
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

In [2]:
path = "../many_names_collection/proc_data_phase0/useful_dataframes/all_responses_round0-3_analyzed.csv"

In [127]:
resdf = pd.read_csv(path,sep="\t")
resdf['vg_is_max'] = resdf.eval('vg_is_max')
resdf['n_anno'] = resdf.eval('n_anno')
resdf['percent_agree'] = resdf.eval('percent_agree')



In [128]:
#resdf['all_responses'] = resdf['all_responses'].apply(lambda x: Counter(eval(x)))
resdf['spellchecked'] = resdf['spellchecked'].apply(lambda x: Counter(eval(x)))

In [129]:
resdf['mn_obj_name'] = resdf['spellchecked'].apply(lambda x: x.most_common(1)[0][0])

In [130]:
resdf.head()

,Unnamed: 0,Unnamed: 0.1,vg_img_id,cat,synset,vg_obj_name,responses_r0,opt-outs,vg_domain,top_response_domain_r0,...,spellchecked,spellchecked_min3,vg_is_max,vg_mean,vg_overlap,n_anno,n_types,snodgrass,percent_agree,mn_obj_name
0,0,0,2357077,person,athlete.n.01,batter,"{'man': 4, 'batter': 2, 'baseball player': 1, ...",{},people,people,...,"{'man': 11, 'batter': 8, 'player': 5, 'person'...","Counter({'man': 11, 'batter': 8, 'player': 5, ...",0,0.250000,8,35,9,2.681758,0.314286,man
1,1,1,2378759,article of clothing,overgarment.n.01,jacket,"{'jacket': 3, 'sweatshirt': 2, 'man': 2, 'hood...",{},clothing,clothing,...,"{'jacket': 14, 'man': 11, 'sweatshirt': 4, 'ho...","Counter({'jacket': 14, 'man': 11, 'sweatshirt'...",1,0.400000,14,36,6,2.132238,0.388889,jacket
2,2,2,2319953,animal,ungulate.n.01,giraffe,{'giraffe': 7},{'#bbox': 2},animals_plants,animals_plants,...,"{'giraffe': 32, 'woman': 1}",Counter({'giraffe': 32}),1,1.000000,32,33,2,0.195909,0.969697,giraffe
3,3,3,2361401,animal,ungulate.n.01,giraffe,{'giraffe': 9},{},animals_plants,animals_plants,...,{'giraffe': 36},Counter({'giraffe': 36}),1,1.000000,36,36,1,0.000000,1.000000,giraffe
4,4,4,2401994,"food, nutrient",dish.n.02,sandwich,"{'sandwich': 6, 'food': 2, 'hotdog': 1}",{},food,food,...,"{'sandwich': 26, 'food': 7, 'hotdog': 1, 'cake...","Counter({'sandwich': 26, 'food': 7})",1,0.787879,26,36,5,1.229288,0.722222,sandwich


In [132]:
# agree_t : agreement threshold
# anno_t : threshold for number of annotations
def get_n_names(resdf, agree_t, anno_t, vg_is_max=False):
    print("\n\nFor images with ")
    print("- an agreement threshold of", agree_t)
    print("- %d annotations per minimum"%anno_t)
    
    q = 'percent_agree >= %.2f & n_anno > %d'%(agree_t,anno_t)
    if vg_is_max:
        q += ' & vg_is_max == 1'
    perfectdf = resdf.query(q)
    names = Counter(list(perfectdf['mn_obj_name']))
    synsets = Counter(list(perfectdf['synset']))
    print("... there is high agreement for: ")
    print(" - %d instances"%len(perfectdf))
    print(" - %d different names (types)"%len(names))
    print(" - %d different synsets "%len(synsets))
    
    print(names.most_common(10))
    print(names.most_common()[-10:])
    

In [133]:
get_n_names(resdf,1.0,33)



For images with 
- an agreement threshold of 1.0
- 33 annotations per minimum
... there is high agreement for: 
 - 1329 instances
 - 33 different names (types)
 - 22 different synsets 
[('train', 251), ('giraffe', 208), ('cat', 176), ('horse', 145), ('zebra', 118), ('dog', 118), ('bed', 59), ('cake', 53), ('cow', 34), ('pizza', 28)]
[('spoon', 1), ('fork', 1), ('curtain', 1), ('bowl', 1), ('tent', 1), ('kite', 1), ('elephant', 1), ('table', 1), ('hat', 1), ('clock', 1)]


In [134]:
get_n_names(resdf,1.0,33, vg_is_max=True)



For images with 
- an agreement threshold of 1.0
- 33 annotations per minimum
... there is high agreement for: 
 - 1294 instances
 - 29 different names (types)
 - 20 different synsets 
[('train', 231), ('giraffe', 208), ('cat', 176), ('horse', 143), ('zebra', 116), ('dog', 115), ('bed', 59), ('cake', 53), ('cow', 33), ('pizza', 27)]
[('car', 5), ('shirt', 2), ('donut', 2), ('vase', 2), ('spoon', 1), ('fork', 1), ('bowl', 1), ('tent', 1), ('table', 1), ('hat', 1)]


In [135]:
get_n_names(resdf,0.9,33)



For images with 
- an agreement threshold of 0.9
- 33 annotations per minimum
... there is high agreement for: 
 - 4988 instances
 - 69 different names (types)
 - 38 different synsets 
[('train', 805), ('giraffe', 505), ('horse', 465), ('cat', 406), ('bed', 399), ('dog', 351), ('pizza', 250), ('zebra', 245), ('bench', 174), ('cake', 173)]
[('butterfly', 1), ('pancake', 1), ('cup', 1), ('kite', 1), ('duck', 1), ('elephant', 1), ('castle', 1), ('mirror', 1), ('window', 1), ('clock', 1)]


In [136]:
get_n_names(resdf,0.9,33,vg_is_max= True)



For images with 
- an agreement threshold of 0.9
- 33 annotations per minimum
... there is high agreement for: 
 - 4773 instances
 - 58 different names (types)
 - 36 different synsets 
[('train', 724), ('giraffe', 504), ('horse', 461), ('cat', 405), ('bed', 394), ('dog', 341), ('zebra', 243), ('pizza', 227), ('bench', 174), ('cake', 170)]
[('salad', 2), ('soup', 2), ('grill', 2), ('dugout', 1), ('cookie', 1), ('butterfly', 1), ('pancake', 1), ('cup', 1), ('duck', 1), ('castle', 1)]


In [137]:
get_n_names(resdf,0.8,30)



For images with 
- an agreement threshold of 0.8
- 30 annotations per minimum
... there is high agreement for: 
 - 10013 instances
 - 112 different names (types)
 - 49 different synsets 
[('train', 1023), ('giraffe', 818), ('horse', 761), ('bed', 707), ('cat', 683), ('dog', 617), ('bench', 488), ('pizza', 469), ('zebra', 396), ('shirt', 304)]
[('tree', 1), ('ramp', 1), ('fruit', 1), ('hose', 1), ('monkey', 1), ('castle', 1), ('stroller', 1), ('clock', 1), ('tomato', 1), ('keyboard', 1)]


In [138]:
get_n_names(resdf,0.8,30,vg_is_max=True)



For images with 
- an agreement threshold of 0.8
- 30 annotations per minimum
... there is high agreement for: 
 - 9261 instances
 - 83 different names (types)
 - 42 different synsets 
[('train', 887), ('giraffe', 816), ('horse', 753), ('bed', 698), ('cat', 681), ('dog', 589), ('bench', 487), ('pizza', 395), ('zebra', 393), ('shirt', 284)]
[('bucket', 1), ('wheelchair', 1), ('pancake', 1), ('pickle', 1), ('dolphin', 1), ('garage', 1), ('hose', 1), ('building', 1), ('castle', 1), ('tomato', 1)]


In [139]:
get_n_names(resdf,0.7,30)



For images with 
- an agreement threshold of 0.7
- 30 annotations per minimum
... there is high agreement for: 
 - 12755 instances
 - 142 different names (types)
 - 51 different synsets 
[('train', 1079), ('giraffe', 819), ('bed', 794), ('horse', 774), ('cat', 701), ('dog', 630), ('bench', 604), ('pizza', 551), ('shirt', 550), ('woman', 438)]
[('sand', 1), ('cheetah', 1), ('bun', 1), ('stroller', 1), ('rose', 1), ('meat', 1), ('tomato', 1), ('flamingo', 1), ('keyboard', 1), ('aquarium', 1)]


In [140]:
get_n_names(resdf,0.6,30)



For images with 
- an agreement threshold of 0.6
- 30 annotations per minimum
... there is high agreement for: 
 - 16140 instances
 - 190 different names (types)
 - 52 different synsets 
[('train', 1115), ('bed', 854), ('giraffe', 820), ('woman', 808), ('shirt', 789), ('horse', 784), ('cat', 708), ('man', 679), ('bench', 650), ('dog', 633)]
[('rose', 1), ('carriage', 1), ('meat', 1), ('tomato', 1), ('bar', 1), ('keyboard', 1), ('church', 1), ('hand', 1), ('wall', 1), ('backpack', 1)]


In [29]:
resdf['all_responses']

0        Counter({'man': 11, 'batter': 8, 'player': 5, ...
1        Counter({'jacket': 14, 'man': 11, 'sweatshirt'...
2                     Counter({'giraffe': 32, 'woman': 1})
3                                 Counter({'giraffe': 36})
4        Counter({'sandwich': 25, 'food': 6, 'hotdog': ...
5        Counter({'van': 15, 'car': 10, 'automobile': 3...
6        Counter({'car': 33, 'automobile': 1, 'vehicle'...
7        Counter({'desk': 29, 'keyboard': 3, 'curtain':...
8                       Counter({'zebra': 33, 'horse': 1})
9        Counter({'child': 14, 'boy': 11, 'kid': 3, 'pe...
10       Counter({'woman': 26, 'girl': 2, 'mom': 1, 'sw...
11       Counter({'jacket': 22, 'coat': 9, 'clothing': ...
12                Counter({'cat': 33, 'cow': 1, 'dog': 1})
13       Counter({'truck': 24, 'garbage truck': 5, 'tra...
14       Counter({'pizza': 31, 'pizza toppings': 1, 'sa...
15                     Counter({'bed': 33, 'mattress': 1})
16                                 Counter({'bridge': 35

In [14]:
33/36

0.9166666666666666